Minimum Spanning Tree

In [ ]:
# number og edges = number of vertices-1
# all nodes are reachable , ie no splitted components

# if a normal tree is converted to spanning tree then numbe of possible spanning trees  = 
# ((E)C(E-1)) - (number of loops)

# for a given weighted graph , the spanning tree with min weights sum is callled minimum spanning tree

Prim's Algorithm

In [ ]:
import heapq
class Solution:
    def prims(self,graph):
        visited  = set()
        mst = []
        heap = []
        heapq.heappush(heap,(0,0,-1))

        while heap:
            print(heap)
            print(mst)
            print(visited)
            print("******************")
            # print(len(mst),len(graph))
            weight,node,parent = heapq.heappop(heap)
            if node in visited:
                print(node,"*********")
                continue
            mst.append((weight,node,parent))
            visited.add(node)
            for nei,nei_w in graph[node]:
                if nei not in visited:
                    heapq.heappush(heap,(nei_w,nei,node))

        return mst,sum([i for i,_,_ in mst])
# o(Elog(v))
# O(V+E)
graph = [
    [(1,2),(2,1)],
    [(0,2),(2,1)],
    [(0,1),(1,1),(4,2),(3,2)],
    [(4,1)],
    [(2,2),(3,1)]
]
Solution().prims(graph)


Disjoint Set | Union by Rank 

In [ ]:
# the core idea is to make connected a order of (1) operation
class DisjointSet:
    def __init__(self, n):
        self.parent = [i for i in range(n + 1)]
        self.rank = [0] * (n + 1)

    def find(self, node):
        if self.parent[node] != node:
            self.parent[node] = self.find(self.parent[node])  # path compression
        return self.parent[node]

    def union(self, u, v):
        pu = self.find(u)
        pv = self.find(v)
        if pu == pv:
            return  # already in same set

        if self.rank[pu] < self.rank[pv]:
            self.parent[pu] = pv
        elif self.rank[pv] < self.rank[pu]:
            self.parent[pv] = pu
        else:
            self.parent[pv] = pu
            self.rank[pu] += 1

    def connected(self, u, v):
        return self.find(u) == self.find(v)


Disjoint Set | Union by size

In [ ]:
# the core idea is to make connected a order of (1) operation
# assuming thee graph given is zero indexed
class DisjointSet:
    def __init__(self, n):
        self.parent = [i for i in range(n)]
        self.size = [1] * (n)

    def find(self, node):
        if self.parent[node] != node:
            self.parent[node] = self.find(self.parent[node])  # path compression
        return self.parent[node]

    def union(self, u, v):
        pu = self.find(u)
        pv = self.find(v)
        if pu == pv:
            return False # already in same set

        if self.size[pu] < self.size[pv]:
            self.parent[pu] = pv
            self.size[pv] += self.size[pu]

        elif self.size[pv] <= self.size[pu]:
            self.parent[pv] = pu
            self.size[pu] += self.size[pv]
        return True
        # else:
        #     self.parent[pv] = pu
        #     self.size[pu] += 1

    def connected(self, u, v):
        return self.find(u) == self.find(v)



kruskal

In [ ]:
import heapq
class Solution:
    def kruskal(self,graph):
        mst = []
        heap = []
        for u in range(len(graph)):
            for v,w in graph[u]:
                if u < v:   # prevent duplicate edges in case of UG
                    heapq.heappush(heap,(w,u,v))
        disjoin = DisjointSet(len(graph))
        while heap:
            w,u,v = heapq.heappop(heap)
            if disjoin.connected(u,v):
                continue
            disjoin.union(u,v)
            mst.append((w,u,v))

        return mst,sum([i for i,_,_ in mst])



graph = [
    [(1,2),(2,1)],
    [(0,2),(2,1)],
    [(0,1),(1,1),(4,2),(3,2)],
    [(4,1)],
    [(2,2),(3,1)]
]
Solution().kruskal(graph)

Number of Operations to Make Network Connected


In [ ]:
'''
1) count the extra edges that u have 
extra = num_of_edges_currently - num_of_edges_in_mst(ie v-1)

2) to connect n components u need n-1 edges, so count the number of components in given network 

3) if num_components-1 < extra then return num_components-1 else it is not possible to connect entire network because we are falling short of cables
'''
class Solution(object):
    def makeConnected(self, n, connections):
        connections = connections[:]
        """
        :type n: int
        :type connections: List[List[int]]
        :rtype: int
        """
        
        ds = DisjointSet(n)
        extra = 0
        for i,j in connections:
            if ds.connected(i,j)==False:
                ds.union(i,j)
            else:
                extra+=1
        
        ultimate_parent = set()
        components = 0
        for i in range(n):
            if ds.find(i) not in ultimate_parent:
                ultimate_parent.add(ds.find(i))
                components+=1
        required_edges = components-1
        
        if extra<required_edges:
            return -1
        else:
            return components-1
# o(V+E)
# o(V)
Solution().makeConnected( n = 4, connections = [[0,1],[0,2],[1,2]])


Most Stones Removed with Same Row or Column

In [ ]:
'''
main idea ->the stones_array can be thought as the nodes that share same x or y coordinate are connected to each other
1) we need to create a dsu first of all , so define an offset so that we can store x and y values seperately selse connecting 1,1 would connect a node to itsely but we will do union of 1,1+offset this will connect x coordiate to y coordinate
2)now if suppose we encounter an x coordinate later on whose y coordinate was encountered earlier so that new x coordinate will get connected to same y coordinate and hence the new coordinate is now connected to old one creating correct DSU
2)now check the distinct components present
3) stones_left_at_end = num of distict components because only one stone will be left per componet(remember tht we are supposed to maximise the number of removals)
'''
class Solution:
    def removeStones(self, stones):
        OFFSET = max(set(x for sublist in stones for x in sublist))+1 # based onconstraints mentioned on leetcode
        ds = DisjointSet(2*OFFSET+1)

        for x,y in stones:
            ds.union(x,y+OFFSET)

        # track all unique roots that are actually used
        roots = set()
        for x,y in stones:
            roots.add(ds.find(x))
            # roots.add(ds.find(y+OFFSET))

        return len(stones)-len(roots)

# o(n)
# o(n)
      
Solution().removeStones([[0,0],[2,2],[10000,2]])

        

Accounts Merge

In [ ]:
from collections import defaultdict
class Solution(object):
    def accountsMerge(self, accounts):
        """
        :type accounts: List[List[str]]
        :rtype: List[List[str]]
        """
        n = len(accounts)
        dic = {}
        ds = DisjointSet(n)

        for i,acc in enumerate(accounts):
            for email in acc[1:]:
                if email in dic:
                    ds.union(dic[email],i)
                dic[email] = i

        print(dic)

        merged = defaultdict(set)
        for email,idx in dic.items():
            parent = ds.find(idx)
            merged[parent].add(email)
        
        result = []
        for parent,emails in merged.items():
            name = accounts[parent][0]
            result.append([name]+sorted(list(emails)))
          
        
        return result

# o(n*m*log(m*n)) where n is number of accounts and m is avg_number of accounts per user
# o(n*m)
        
Solution().accountsMerge(accounts = [["John","johnsmith@mail.com","john_newyork@mail.com"],["John","johnsmith@mail.com","john00@mail.com"],["Mary","mary@mail.com"],["John","johnnybravo@mail.com"]])


Island 2

In [ ]:
'''
* Each land addition represents a new potential island.
* We use a (DSU) to efficiently track how separate land pieces connect.
* Each cell in the grid is represented by a unique ID:
  → `id = row * m + col`
* Maintain a `visited` set for cells that have turned into land.
* Initialize `count = 0` (current number of islands) and an empty list `ans` for results.

* For each land addition `(x, y)`:

  * If already visited → append the previous count (no change) and continue.
  else:
  * Mark `(x, y)` as visited and assume it forms a new island → `count += 1`.
  * For each of the four directions `(±1, 0)` and `(0, ±1)`:

    * Compute neighbor coordinates `(nx, ny)`.
    * Skip if out of grid bounds.
    * If neighbor is visited (i.e.,this is a land):

      * Get both roots using DSU: `root_curr = ds.find(curr)` and `root_nei = ds.find(nei)`.
      * **If roots differ**, they belong to different islands →
        perform `ds.union(curr, nei)` and **decrease** island count by 1.
      * **If roots are same**, they’re already part of one island → skip.
  * Append the current `count` to `ans`.

**Key Insights:**

* Each new land starts as an individual island.
* Union merges adjacent lands into a single island.
* Decrement happens **only when merging two distinct components**, preventing double subtraction.
* The condition `if ds.find(nei) != ds.find(curr)` ensures we merge only once per unique neighbor island.
* The DSU efficiently handles connected components with near O(1) operations.

**Result:**

* The `ans` list records the total number of islands after each operation.

'''
class Solution:
    def islands(self,n,m,k,arr):
        ds = DisjointSet(n*m)
        delta = [(1,0),(0,1),(-1,0),(0,-1)]
        ans = []
        visited = set()

        for i in range(k):
            x,y = arr[i]
            item = x*m+y
            if item in visited:
                ans.append(ans[-1])
                continue

            visited.add(item)
            comp = ans[-1]+1 if ans else 1

            for di,dj in delta:
                nei_x = x+di
                nei_y = y+dj

                if 0<=nei_x<n and 0<=nei_y<m :
                    nei = nei_x*m+nei_y
                    if nei  in visited:
                        print(nei,x,y,di,dj)
                        if ds.find(nei) != ds.find(item):
                            ds.union(nei,item)
                            comp-=1
            ans.append(comp)
        return ans

Solution().islands(n = 4, m = 5, k = 12, arr= [[0,0],[0,0],[1,1],[1,0],[0,1],[0,3],[1,3],[0,4], [3,2], [2,2],[1,2], [0,2]]  )

Making A Large Island

In [117]:
'''
In the first traversal of the grid, the goal is to unite all adjacent land cells (1s) using Disjoint Set Union (DSU) by size.
This step ensures that every connected island is treated as one single component with its total area stored in its root’s size.

In the second traversal, we only consider water cells (0s).
For each such cell, check its four neighboring directions (up, down, left, right) and collect the unique island roots it touches.
Then, sum up the sizes of these distinct neighboring islands and add 1 (for the current cell that will be flipped from 0 → 1).
Append this total to an ans array.

Finally, return the maximum value from the ans array — this represents the largest possible island size that can be formed by flipping exactly one 0 into 1.
'''
class Solution:
    def largestIsland(self, grid):
        n = len(grid)
        ds = DisjointSet(n*n)
        delta = [(1,0),(0,1),(-1,0),(0,-1)]

        for i in range(n):
            for j in range(n):
                if grid[i][j]==0:
                    continue
                item = i*n+j
                
                for di,dj in delta:
                    row,col = i+di,j+dj
                    if 0<=row<n and 0<=col<n and grid[row][col]==1:
                        nei = row*n+col
                        ds.union(nei,item)
        print(ds.size)
        ans = 0
        has_zero = False
        
        for i in range(n):
            for j in range(n):
                if grid[i][j]==0:
                    has_zero = True
                    item = i*n+j
                    nei_sizes = 0
                    vis = set()

                    for di,dj in delta:
                        row,col = i+di,j+dj
                        nei = row*n+col
                        if 0<=row<n and 0<=col<n and ds.find(nei) not in vis and grid[row][col]==1:
                            vis.add(ds.find(nei))
                            nei_sizes += ds.size[ds.find(nei)]
                    ans = max(ans,nei_sizes+1)


        if not has_zero:
            return n * n
        return ans
# o(m*n*4)
# o(m*n)
Solution().largestIsland(grid = [[1,0],[0,1]])

                    


[1, 1, 1, 1]


3

Swim in Rising Water

In [134]:
'''
what we are basically doing is we reach a node , it is guaranteed in dijkstra that we have reached the current node using the best way possible thats why it was popped out first and just as it was popped we marked it as visited so the ways to reach that node in suboptimal ways will not be processed  in future, now moving forward towards neighbor the new elevation will be max_till_now   or the elevation of neighbor whichone is greater, we push the max of both with new coordinates into the heap , if somehow we find a better way to reach that nei with less time then that time will automatically come above in heap and thats the reason we are making nodes as visited after popping not at pushing
'''
import heapq
class Solution:
    def swimInWater(self, grid):
        heap = []
        n = len(grid)
        visited = set()
        heapq.heappush(heap,(grid[0][0],0,0))

        while heap:
            time,i,j = heapq.heappop(heap)
            if (i,j)==(n-1,n-1):
                return time
            if (i,j) in visited:
                continue
            visited.add((i,j))

            for di,dj in [(0,1),(1,0),(-1,0),(0,-1)]:
                row,col = i+di,j+dj
                if 0<=row<n and 0<=col<n and (row,col) not in visited:
                    new_time = max(time,grid[row][col])
                    heapq.heappush(heap,(new_time,row,col))


# o(n*n*logn)
# o(n*n)
Solution().swimInWater(grid = [[3,2],[0,1]])


3

In [ ]:
'''
since we are tring to minimize max_edge_weight in process of reacing from 0,0 to -1,-1 , this problem can also we solved using kruskal mst algorithm

Think of the water rising level by level:
At each new elevation, more adjacent land cells get connected.
DSU keeps track of which areas are now reachable underwater.
The moment your start cell and destination cell belong to the same flooded region → you can swim between them.
That current elevation = answer.

“We process edges in increasing elevation order using DSU to simulate the water rising. Each union represents new connectivity between cells. The moment the start and end cells belong to the same connected component, that elevation is the minimum water level required to reach the destination.”
'''
import heapq
class Solution:
    def swimInWater(self, grid):
        n = len(grid)
        edges = []

        def index(i,j):
            return i*n+j
        
        # Step 1: Build edges between all 4-directional neighbors
        for i in range(n):
            for j in range(n):
                if i+1<n: #checks if there’s a cell below the current one (downward neighbor).
                    nei_w = max(grid[i][j],grid[i+1][j]) #Because in this problem, to “swim” between two cells, the water must rise to at least the higher of the two elevations.
                    heapq.heappush(edges,((nei_w,index(i,j),index(i+1,j))))
                if j+1<n: #checks if there’s a cell to the right of the current one (rightward neighbor).
                    nei_w = max(grid[i][j],grid[i][j+1])
                    heapq.heappush(edges,((nei_w,index(i,j),index(i,j+1))))

        #After this loop, edges contains all possible connections between adjacent cells (right and bottom neighbors (this is an UG so exploring these 2 directions is enough)).
        # Each edge’s weight equals the maximum elevation of the two cells it connects — representing the minimum water level needed to traverse between them.

        # Now we process edges in ascending order of elevation (the “water level”).
        ds = DisjointSet(n*n)
        while edges:
            elevation,u,v = heapq.heappop(edges)
            # So conceptually, at each iteration:-“Water rises to height = elevation, connecting these two land cells.”
            if ds.connected(u,v):
                continue
            ds.union(u,v) #We merge the two components (regions) because now, at this elevation, water connects them.
            if ds.find(0) == ds.find(n*n-1): #“If the start and destination cells are now part of the same connected water region, the water level (elevation) at which this happened is the minimum time required to swim from start to end.” 
            # minimum here is ensured by using heapq
                return elevation